Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [155]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import numpy as np


## The *Nim* and *Nimply* classes

In [156]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [157]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self): #checks if there are objects in any row
        return sum(self._rows) > 0

    def __str__(self): #string for current state
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property 
    def rows(self) -> tuple: #returns a tuple with the current number of objects in each row
        return tuple(self._rows)

    #is used to make a move in the game by specifying a row and the number of objects to remove
    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [158]:
#random move

def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [159]:

#maximum possible in the lowest row

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [160]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}
    
   # Extract the "love_small" parameter from the genome
    love_small_param = genome["love_small"]
    
   
    # Calculate some values ​​based on the current state of the game
    # Count the number of active rows, i.e. the number of rows still containing objects.
    active_rows_number = sum(o > 0 for o in state.rows)
    
    # Find the shortest row, i.e. the one with the fewest objects.
    # The lambda function is used as the key to determine the MINIMUM based on the number of objects.
    shortest_row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    
    # Find the longest line, i.e. the one with the most objects.
    # The lambda function is used as the key to determine the MAXIMUM based on the number of objects.
    longest_row = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    
    #Make a decision based on the "love_small" parameter
    if love_small_param > 0.5:
        #If "love_small" is greater than 0.5, choose the smallest move on the shortest row
        ply = Nimply(shortest_row, 1) 
    else:
        # Otherwise, choose the largest move on the longest row
        ply = Nimply(longest_row, state.rows[longest_row])
    
    return ply

#Instead of explicitly computing these values, the strategy randomly selects a non-empty row
# and make a move based on the chosen row.
# This simplifies the strategy while still incorporating the adaptive parameter.

def adaptive_alternativa(state: Nim) -> Nimply:
    genome = {"love_small": 0.5}
    
    # Extract the "love_small" parameter from the genome
    love_small_param = genome["love_small"]
    
    
    # Calculate some values ​​based on the current state of the game        
    active_rows = [i for i, oggetti in enumerate(state.rows) if oggetti > 0]
    
    
    # Make a decision based on the parameter and state of the game  
    if love_small_param > 0.5:
        # Choose the smallest move on a non-empty row at random
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, 1)
    else:
        # Choose the largest move on a non-empty row at random
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, state.rows[fila_scelta])
    
    return ply

def adaptive_dynamic(state: Nim) -> Nimply:
    genome = {"love_small": 0.5}
    
    # Extract the "love_small" parameter from the genome
    love_small_param = genome["love_small"]
    
    
    # Calculate some values ​​based on the current state of the game
    active_rows = [i for i, objects in enumerate(state.rows) if objects > 0]
    
    # Make adaptive parameters more dynamic based on the percentage of empty lines compared to the total lines of the game.
    #so that it is proportional to the amount of space available in the game.
    # Essentially, if there are more empty lines, love_small_param will be affected more, and vice versa.
    love_small_param *= state.rows.count(0) / len(state.rows)
    
    
    # Make a decision based on the parameter and state of the game
    if love_small_param > 0.5:
        # Choose the smallest move on a non-empty row at random
        choice_row = random.choice(active_rows)
        ply = Nimply(choice_row, 1)
    else:
        # Choose the largest move on a non-empty row at random
        choice_row = random.choice(active_rows)
        ply = Nimply(choice_row, state.rows[choice_row])
    
    return ply





Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [ ]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import numpy as np


## The *Nim* and *Nimply* classes

In [ ]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [ ]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self): #checks if there are objects in any row
        return sum(self._rows) > 0

    def __str__(self): #string for current state
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property 
    def rows(self) -> tuple: #returns a tuple with the current number of objects in each row
        return tuple(self._rows)

    #is used to make a move in the game by specifying a row and the number of objects to remove
    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [ ]:
#random move

def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [ ]:

#maximum possible in the lowest row

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [ ]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}
    
   # Extract the "love_small" parameter from the genome
    love_small_param = genome["love_small"]
    
   
    # Calculate some values ​​based on the current state of the game
    # Count the number of active rows, i.e. the number of rows still containing objects.
    active_rows_number = sum(o > 0 for o in state.rows)
    
    # Find the shortest row, i.e. the one with the fewest objects.
    # The lambda function is used as the key to determine the MINIMUM based on the number of objects.
    shortest_row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    
    # Find the longest line, i.e. the one with the most objects.
    # The lambda function is used as the key to determine the MAXIMUM based on the number of objects.
    longest_row = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    
    #Make a decision based on the "love_small" parameter
    if love_small_param > 0.5:
        #If "love_small" is greater than 0.5, choose the smallest move on the shortest row
        ply = Nimply(active_rows_number, 1) 
    else:
        # Otherwise, choose the largest move on the longest row
        ply = Nimply(longest_row, state.rows[longest_row])
    
    return ply

#Instead of explicitly computing these values, the strategy randomly selects a non-empty row
# and make a move based on the chosen row.
# This simplifies the strategy while still incorporating the adaptive parameter.

def adaptive_alternativa(state: Nim) -> Nimply:
    genome = {"love_small": 0.5}
    
    # Estrai il parametro "love_small" dal genoma
    love_small_param = genome["love_small"]
    
    # Calcola alcuni valori basati sullo stato attuale del gioco
    active_rows = [i for i, oggetti in enumerate(state.rows) if oggetti > 0]
    
    # Prendi una decisione basata sul parametro e lo stato del gioco
    if love_small_param > 0.5:
        # Scegli la mossa più piccola su una fila non vuota a caso
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, 1)
    else:
        # Scegli la mossa più grande su una fila non vuota a caso
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, state.rows[fila_scelta])
    
    return ply

def adaptive_dynamic(state: Nim) -> Nimply:
    genome = {"love_small": 0.5}
    
    # Estrai il parametro "love_small" dal genoma
    love_small_param = genome["love_small"]
    
    # Calcola alcuni valori basati sullo stato attuale del gioco
    active_rows = [i for i, oggetti in enumerate(state.rows) if oggetti > 0]
    
    # Rendi i parametri adattivi più dinamici  in base alla percentuale di righe vuote rispetto al totale delle righe del gioco.
    #n modo che sia proporzionale alla quantità di spazio disponibile nel gioco. 
    # In sostanza, se ci sono più righe vuote, love_small_param sarà influenzato di più, e viceversa.
    love_small_param *= state.rows.count(0) / len(state.rows)
    
    # Prendi una decisione basata sul parametro e lo stato del gioco
    if love_small_param > 0.5:
        # Choose the smallest move on a non-empty row at random
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, 1)
    else:
        # Choose the largest move on a non-empty row at random
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, state.rows[fila_scelta])
    
    return ply





In [ ]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

#analyzes the current state and returns the possible mods and the final result sum after each move
def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


#search for the best moves based on the possible moves
def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [ ]:
logging.getLogger().setLevel(logging.INFO)
#switch between strategies based on those chosen by the players
strategy = (adaptive_dynamic, pure_random)

nim = Nim(7)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9 11 13>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=7)
INFO:root:status: <1 3 5 0 9 11 13>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 4 0 9 11 13>
INFO:root:ply: player 0 plays Nimply(row=5, num_objects=11)
INFO:root:status: <1 3 4 0 9 0 13>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=2)
INFO:root:status: <1 3 2 0 9 0 13>
INFO:root:ply: player 0 plays Nimply(row=6, num_objects=13)
INFO:root:status: <1 3 2 0 9 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 1 0 9 0 0>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=9)
INFO:root:status: <1 3 1 0 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 0 0 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 0 0 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 2 0 0 0 0 0>
INFO:root:ply: player 0 pla

In [161]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

#analyzes the current state and returns the possible mods and the final result sum after each move
def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


#search for the best moves based on the possible moves
def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [162]:
logging.getLogger().setLevel(logging.INFO)
#switch between strategies based on those chosen by the players
strategy = (adaptive_dynamic, pure_random)

nim = Nim(7)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9 11 13>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=7)
INFO:root:status: <1 3 5 0 9 11 13>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 4 0 9 11 13>
INFO:root:ply: player 0 plays Nimply(row=5, num_objects=11)
INFO:root:status: <1 3 4 0 9 0 13>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=2)
INFO:root:status: <1 3 2 0 9 0 13>
INFO:root:ply: player 0 plays Nimply(row=6, num_objects=13)
INFO:root:status: <1 3 2 0 9 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 1 0 9 0 0>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=9)
INFO:root:status: <1 3 1 0 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 0 0 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 0 0 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 2 0 0 0 0 0>
INFO:root:ply: player 0 pla